The main objective of the following article is to obtain a step-by-step guide on how to forecast with `CatboostRegressor` for a *time series model with machine learning* using `Mlforecast`.

During this walkthrough, we will become familiar with the main `MlForecast` class and some relevant methods such as `mlforecast.fit`, `mlforecast.predict` and `mlforecast.cross_validation` in other.

Let's start!!!

<a class="anchor" id="0.1"></a>



1.	[Introduction](#1)
2.	[What is CatBoostRegressor?](#2)
3.	[Loading libraries and data](#3)
4.	[Explore Data with the plot method](#4)
5.	[Modeling with MLForecast](#5)
6.  [Feature importances](#6)
7.  [Evaluate the model’s performance](#7)
8.  [Evaluate the model](#8)
9.  [References](#9)

# **Introduction** <a class="anchor" id="1"></a>

[Table of Contents](#0.1)

`CatBoostRegressor` is a machine learning library developed by Yandex used for regression problems. It is an implementation of `Gradient Boosting` that uses decision trees as base estimators. What sets `CatBoostRegressor` apart is its ability to handle categorical features automatically and efficiently, without the need for manual coding.

`CatBoostRegressor` is designed to deliver fast and efficient performance, allowing you to train models on large data sets and make real-time predictions. It also offers features such as interaction modeling and missing data handling, which helps improve the accuracy and generalizability of the model.

Some of the advantages of `CatBoostRegressor` include its ability to deal with categorical features without manual coding, its ability to capture interactions between features, its handling of missing data, and its ability to avoid overfitting using regularization techniques.

# **What is CatBoostRegressor?** <a class="anchor" id="2"></a>
[Table of Contents](#0.1)

CatBoost is an open source library developed by Yandex, a Russian technology company, that offers tree boosting machine learning algorithms. Specifically, CatBoost is known for its ability to handle categorical features efficiently and automatically, making it especially suitable for data sets containing categorical variables.

CatBoost's boosting algorithms are based on the Gradient Boosting technique, which combines multiple weak estimators (in this case, decision trees) to create a stronger, more accurate model. Some key features and benefits of CatBoost are:

1. **Automatic handling of categorical features**: CatBoost can automatically handle categorical features without requiring manual coding. It uses a coding algorithm based on the order of responses and the number of observations in each category.

2. **Interaction Modeling**: CatBoost automatically captures interactions between features, which can help improve model performance.

3. **Handling Missing Data**: CatBoost has the ability to handle missing values in input data. There is no need to perform imputations or remove rows with missing values before training the model.

4. **Advanced Regularization**: CatBoost uses advanced regularization techniques, such as L2 regularization, to prevent overfitting and improve the generalization ability of the model.

5. **High speed of training and prediction**: CatBoost is designed for fast and efficient performance, allowing you to train models on large data sets and make fast predictions in real time.

CatBoost supports several interfaces, including Python, R, and the command line. Additionally, it integrates with other popular machine learning libraries, such as scikit-learn and XGBoost, allowing you to take advantage of the additional capabilities of these libraries.

## **Install catboost**

* `pip install catboost`
* `conda install -c conda-forge catboost`

# **Loading libraries and data** <a class="anchor" id="3"></a>

[Table of Contents](#0.1)

In [1]:
# Handling and processing of Data
# ==============================================================================
import numpy as np
import pandas as pd

import scipy.stats as stats

# Handling and processing of Data for Date (time)
# ==============================================================================
import datetime
import time
from datetime import datetime, timedelta

# 
# ==============================================================================
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.seasonal import seasonal_decompose 

In [2]:
# Models 
# ==============================================================================
from catboost import CatBoostRegressor

# MLForecast
# ==============================================================================
from mlforecast import MLForecast
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
from window_ops.ewm import ewm_mean
from mlforecast.target_transforms import Differences
from utilsforecast.plotting import plot_series
from mlforecast.utils import PredictionIntervals
from mlforecast.utils import generate_daily_series, generate_prices_for_series

In [3]:
# Plot
# ==============================================================================
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('grayscale') # fivethirtyeight  grayscale  classic
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#008080',  # #212946
    'axes.facecolor': '#008080',
    'savefig.facecolor': '#008080',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#000000',  #2A3459
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)
# Define the plot size
# ==============================================================================

plt.rcParams['figure.figsize'] = (18,7)

# Hide warnings
# ==============================================================================
import warnings
warnings.filterwarnings("ignore")

## **Read Data**

We will use data from the [City of Los Angeles website traffic dataset](https://www.kaggle.com/datasets/cityofLA/lacity.org-website-traffic) on Kaggle.

This dataset contains the number of user sessions for the City of Los Angeles website for each day from 2014 to 2019.

Sessions are periods of time when a user is active in a website.

In [4]:
df=pd.read_csv("lacity.org-website-traffic.csv")
df.head()

,Date,Device Category,Browser,# of Visitors,Sessions,Bounce Rate
0,2014-01-01T00:00:00.000,desktop,Chrome,900,934,55.5675
1,2014-01-01T00:00:00.000,desktop,Firefox,692,761,40.8673
2,2014-01-01T00:00:00.000,desktop,Internet Explorer,1038,1107,31.2556
3,2014-01-01T00:00:00.000,desktop,Opera,35,35,100.0000
4,2014-01-01T00:00:00.000,desktop,Safari,484,554,24.9097


In [5]:
df.columns = [k.lower().replace(' ', '_').replace('#', 'qnty') for k in df.columns]
df['date'] = pd.to_datetime(df.date).dt.normalize()
df

,date,device_category,browser,qnty_of_visitors,sessions,bounce_rate
0,2014-01-01,desktop,Chrome,900,934,55.5675
1,2014-01-01,desktop,Firefox,692,761,40.8673
2,2014-01-01,desktop,Internet Explorer,1038,1107,31.2556
3,2014-01-01,desktop,Opera,35,35,100.0000
4,2014-01-01,desktop,Safari,484,554,24.9097
...,...,...,...,...,...,...
8348980,2019-08-27,mobile,Chrome,199,318,50.0000
8348981,2019-08-27,mobile,Firefox,40,40,100.0000
8348982,2019-08-27,mobile,Safari,199,199,79.8995
8348983,2019-08-27,tablet,Amazon Silk,40,40,100.0000


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
# Aggregate the data based on Date, Device and Session.
data = df[['date', 'device_category', 'sessions']].groupby(['date', 'device_category']).sum().reset_index()

# Fill Missing Any Date to 0
df_pivot = data.pivot(index='device_category', values=['sessions'], columns='date')
df_pivot.fillna(0, inplace=True)

df_pivot = df_pivot.stack().reset_index().sort_values(by=['date', 'device_category']).reset_index(drop=True)
df_pivot.columns = ['device', 'date', 'session']
data = df_pivot[['date', 'device', 'session']]

del df_pivot

The input to MlForecast is always a data frame in long format with three columns: unique_id, ds and y:

* The `unique_id` (string, int or category) represents an identifier for the series.

* The `ds` (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

* The `y` (numeric) represents the measurement we wish to forecast.

In [8]:
data=data.rename(columns={"date": "ds","device":"unique_id", "session": "y"})
data.head()

,ds,unique_id,y
0,2014-01-01,desktop,616768.0
1,2014-01-01,mobile,287547.0
2,2014-01-01,tablet,15967.0
3,2014-01-02,desktop,2030513.0
4,2014-01-02,mobile,321557.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6195 entries, 0 to 6194
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ds         6195 non-null   datetime64[ns]
 1   unique_id  6195 non-null   object        
 2   y          6195 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 145.3+ KB


# **Explore Data with the plot method** <a class="anchor" id="4"></a>

[Table of Contents](#0.1)

We are going to use the `plot_series` function to visualize our data.

In [11]:
fig = plot_series(data)
fig.savefig('../tutorials/catboost_regressor_forecasting__eda.png')

![](../tutorials/catboost_regressor_forecasting__eda.png)

## **The Augmented Dickey-Fuller Test**
An Augmented Dickey-Fuller (ADF) test is a type of statistical test that determines whether a unit root is present in time series data. Unit roots can cause unpredictable results in time series analysis. A null hypothesis is formed in the unit root test to determine how strongly time series data is affected by a trend. By accepting the null hypothesis, we accept the evidence that the time series data is not stationary. By rejecting the null hypothesis or accepting the alternative hypothesis, we accept the evidence that the time series data is generated by a stationary process. This process is also known as stationary trend. The values of the ADF test statistic are negative. Lower ADF values indicate a stronger rejection of the null hypothesis.

Augmented Dickey-Fuller Test is a common statistical test used to test whether a given time series is stationary or not. We can achieve this by defining the null and alternate hypothesis.

- Null Hypothesis: Time Series is non-stationary. It gives a time-dependent trend.
- Alternate Hypothesis: Time Series is stationary. In another term, the series doesn’t depend on time.

- ADF or t Statistic < critical values: Reject the null hypothesis, time series is stationary.
- ADF or t Statistic > critical values: Failed to reject the null hypothesis, time series is non-stationary.

In [12]:
def augmented_dickey_fuller_test(series , column_name):
    print (f'Dickey-Fuller test results for columns: {column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','No Lags Used','Number of observations used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
        print("Conclusion:====>")
        print("Reject the null hypothesis")
        print("The data is stationary")
    else:
        print("Conclusion:====>")
        print("The null hypothesis cannot be rejected")
        print("The data is not stationary")

In [13]:
augmented_dickey_fuller_test(data["y"],"website-traffic")

Dickey-Fuller test results for columns: website-traffic
Test Statistic                -7.269285e+00
p-value                        1.604152e-10
No Lags Used                   3.400000e+01
Number of observations used    6.160000e+03
Critical Value (1%)           -3.431412e+00
Critical Value (5%)           -2.862009e+00
Critical Value (10%)          -2.567020e+00
dtype: float64
Conclusion:====>
Reject the null hypothesis
The data is stationary


## **Autocorrelation plots**

### **Autocorrelation Function**

**Definition 1.** Let $\{x_t;1 ≤ t ≤ n\}$ be a time series sample of size n from $\{X_t\}$.
1. $\bar x = \sum_{t=1}^n \frac{x_t}{n}$ is called the sample mean of $\{X_t\}$.
2. $c_k =\sum_{t=1}^{n−k} (x_{t+k}- \bar x)(x_t−\bar x)/n$ is known as the sample autocovariance function of $\{X_t\}$.
3. $r_k = c_k /c_0$ is said to be the sample autocorrelation function of $\{X_t\}$. 

Note the following remarks about this definition:
 
* Like most literature, this guide uses ACF to denote the sample autocorrelation function as well as the autocorrelation function. What is denoted by ACF can easily be identified in context.

* Clearly c0 is the sample variance of $\{X_t\}$. Besides, $r_0 = c_0/c_0 = 1$ and for any integer $k, |r_k| ≤ 1$.

* When we compute the ACF of any sample series with a fixed length $n$, we cannot put too much confidence in the values of $r_k$ for large k’s, since fewer pairs of $(x_{t +k }, x_t )$ are available for calculating $r_k$ as $k$ is large. One rule of thumb is not to estimate $r_k$ for $k > n/3$, and another is $n ≥ 50, k ≤ n/4$. In any case, it is always a good idea to be careful.

* We also compute the ACF of a nonstationary time series sample by Definition 1. In this case, however, the ACF or $r_k$ very slowly or hardly tapers off as $k$ increases.

* Plotting the ACF $(r_k)$ against lag $k$ is easy but very helpful in analyzing time series sample. Such an ACF plot is known as a correlogram.

* If $\{X_t\}$ is stationary with $E(X_t)=0$ and $\rho_k =0$ for all $k \neq 0$,thatis,itisa white noise series, then the sampling distribution of $r_k$ is asymptotically normal with the mean 0 and the variance of $1/n$. Hence, there is about 95% chance that $r_k$ falls in the interval $[−1.96/√n, 1.96/√n]$.

Now we can give a summary that (1) if the time series plot of a time series clearly shows a trend or/and seasonality, it is surely nonstationary; (2) if the ACF $r_k$ very slowly or hardly tapers off as lag $k$ increases, the time series should also be nonstationary.

In [14]:
fig, axs = plt.subplots(nrows=1, ncols=2)

plot_acf(data["y"],  lags=30, ax=axs[0],color="fuchsia")
axs[0].set_title("Autocorrelation");

# Grafico
plot_pacf(data["y"],  lags=30, ax=axs[1],color="lime")
axs[1].set_title('Partial Autocorrelation')
plt.savefig("../tutorials/catboost_regressor_forecasting__autocorrelation.png")
plt.close();

![](../tutorials/catboost_regressor_forecasting__autocorrelation.png)

## **Decomposition of the time series**

How to decompose a time series and why?

In time series analysis to forecast new values, it is very important to know past data. More formally, we can say that it is very important to know the patterns that values follow over time. There can be many reasons that cause our forecast values to fall in the wrong direction. Basically, a time series consists of four components. The variation of those components causes the change in the pattern of the time series. These components are:

* **Level:** This is the primary value that averages over time.
* **Trend:** The trend is the value that causes increasing or decreasing patterns in a time series.
* **Seasonality:** This is a cyclical event that occurs in a time series for a short time and causes short-term increasing or decreasing patterns in a time series.
* **Residual/Noise:** These are the random variations in the time series.

Combining these components over time leads to the formation of a time series. Most time series consist of level and noise/residual and trend or seasonality are optional values.

If seasonality and trend are part of the time series, then there will be effects on the forecast value. As the pattern of the forecasted time series may be different from the previous time series.

The combination of the components in time series can be of two types:
* Additive
* multiplicative

### Additive

In [15]:
a = seasonal_decompose(data["y"], model = "additive", period=24).plot()
a.savefig('../tutorials/catboost_regressor_forecasting__seasonal_decompose_aditive.png')
plt.close()

![](../tutorials/catboost_regressor_forecasting__seasonal_decompose_aditive.png)

### Multiplicative

In [16]:
m = seasonal_decompose(data["y"], model = "Multiplicative", period=24).plot()
m.savefig('../tutorials/catboost_regressor_forecasting__seasonal_decompose_multiplicative.png')
plt.close();

![](../tutorials/catboost_regressor_forecasting__seasonal_decompose_multiplicative.png)

# **Modeling with MLForecast** <a class="anchor" id="5"></a>

[Table of Contents](#0.1)

## **Building Model**

We define the model that we want to use, for our example we are going to use the `CatBoostRegressor model`.

In [17]:
models = [CatBoostRegressor(depth=6,
                            iterations=1600,
                            learning_rate=0.024,
                            l2_leaf_reg=20,
                            random_strength=1.5,
                            grow_policy='Depthwise',
                            leaf_estimation_method='Newton', 
                            bootstrap_type='Bernoulli',
                            thread_count=4,
                            verbose=False,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            od_type='Iter',                       
                            early_stopping_rounds=500,
                            random_state=123)]

We fit the models by instantiating a new `MlForecast` object with the following parameters:

* `models:` a list of models. Select the models you want from models and import them.

* `freq:` a string indicating the frequency of the data. (See [panda’s available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

* `lags:` Lags of the target to uses as feature.

* `lag_transforms:` Mapping of target lags to their transformations.

* `date_features:` Features computed from the dates. Can be `pandas` date attributes or functions that will take the dates as input.

* `differences:` Differences to take of the target before computing the features. These are restored at the forecasting step.

* `num_threads:` Number of threads to use when computing the features.

* `target_transforms:` Transformations that will be applied to the target computing the features and restored after the forecasting step.

Any settings are passed into the constructor. Then you call its fit method and pass in the historical data frame.

In [18]:
def even_day(dates):
    return dates.day % 2 == 0

In [19]:
mlf = MLForecast(models=models,
                 freq='D', 
                 lags=[1,7,14],
                 lag_transforms={1: [expanding_mean], 12: [(rolling_mean, 7)] },
                 date_features=['dayofweek', 'month', even_day],
                 num_threads=2,        
                 )

In [20]:
prep = mlf.preprocess(data)
prep

,ds,unique_id,y,lag1,lag7,lag14,expanding_mean_lag1,rolling_mean_lag12_window_size7,dayofweek,month,even_day
54,2014-01-19,desktop,774554.0,600868.0,693748.0,837332.0,1.967281e+06,1.844039e+06,6,1,False
55,2014-01-19,mobile,245317.0,190761.0,289818.0,234696.0,3.679047e+05,3.814936e+05,6,1,False
56,2014-01-19,tablet,17081.0,24028.0,47763.0,87354.0,9.384600e+04,1.029474e+05,6,1,False
57,2014-01-20,desktop,1296779.0,774554.0,2735596.0,2894615.0,1.904506e+06,2.215047e+06,0,1,True
58,2014-01-20,mobile,309914.0,245317.0,152021.0,367005.0,3.614527e+05,4.865529e+05,0,1,True
...,...,...,...,...,...,...,...,...,...,...,...
6190,2019-08-26,mobile,4853.0,4613.0,27757.0,45288.0,5.527932e+05,4.062614e+04,0,8,True
6191,2019-08-26,tablet,199.0,199.0,635.0,397.0,3.374819e+04,3.682857e+02,0,8,True
6192,2019-08-27,desktop,438.0,6440.0,29310.0,70247.0,1.470229e+06,4.671714e+04,1,8,False
6193,2019-08-27,mobile,557.0,4853.0,19782.0,34512.0,5.525277e+05,3.831143e+04,1,8,False


## **Fit method**

The fit method uses the following parameters: 

|Parameters |Type |Default |Details| 
|-----------|-------|-----------|-------| 
|df| DataFrame| |Series data in long format.| 
|target_col |str |y |Column that contains the target. 
|prediction_intervals |typing.Optional[mlforecast.utils.PredictionIntervals] |None |Configuration to calibrate prediction intervals (Conformal Prediction).| 
|fitted |bool |False |Save in-sample predictions.| 


For more information on the parameters: [you can see it here](https://nixtla.github.io/mlforecast/forecast.html#mlforecast.fit)

In [21]:
# fit the models
mlf.fit(data, fitted=True)

MLForecast(models=[CatBoostRegressor], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'expanding_mean_lag1', 'rolling_mean_lag12_window_size7'], date_features=['dayofweek', 'month', <function even_day at 0x16a3fc280>], num_threads=2)

Let's see the results of our model in this case the `CatBoostRegressor model`. We can observe it with the following instruction:

Let us now visualize the fitted values of our models.

In [22]:
result=mlf.forecast_fitted_values()
result=result.set_index("unique_id")
result

,ds,y,CatBoostRegressor
unique_id,,,
desktop,2014-01-19,774554.0,7.841933e+05
desktop,2014-01-20,1296779.0,2.814498e+05
desktop,2014-01-21,2274135.0,8.005038e+04
desktop,2014-01-22,2259527.0,1.645322e+06
desktop,2014-01-23,1962229.0,3.876937e+05
...,...,...,...
tablet,2019-08-23,279.0,8.153519e+04
tablet,2019-08-24,438.0,2.039300e+04
tablet,2019-08-25,199.0,6.644634e+04


In [23]:
from statsmodels.stats.diagnostic import normal_ad
from scipy import stats

sw_result = stats.shapiro(result["CatBoostRegressor"])
ad_result = normal_ad(np.array(result["CatBoostRegressor"]), axis=0)
dag_result = stats.normaltest(result["CatBoostRegressor"], axis=0, nan_policy='propagate')

It's important to note that we can only use this method if we assume that the residuals of our validation predictions are normally distributed. To see if this is the case, we will use a PP-plot and test its normality with the Anderson-Darling, Kolmogorov-Smirnov, and D’Agostino K^2 tests.

The PP-plot(Probability-to-Probability) plots the data sample against the normal distribution plot in such a way that if normally distributed, the data points will form a straight line.

The three normality tests determine how likely a data sample is from a normally distributed population using p-values. The null hypothesis for each test is that "the sample came from a normally distributed population". This means that if the resulting p-values are below a chosen alpha value, then the null hypothesis is rejected. Thus there is evidence to suggest that the data comes from a non-normal distribution. For this article, we will use an Alpha value of 0.01.

In [24]:
result=mlf.forecast_fitted_values()
fig, axs = plt.subplots(nrows=2, ncols=2)

# plot[1,1]
result["CatBoostRegressor"].plot(ax=axs[0,0])
axs[0,0].set_title("Residuals model");

# plot
#plot(result["XGBRegressor"], ax=axs[0,1]);
axs[0,1].hist(result["CatBoostRegressor"], density=True,bins=50, alpha=0.5 )
axs[0,1].set_title("Density plot - Residual");

# plot
stats.probplot(result["CatBoostRegressor"], dist="norm", plot=axs[1,0])
axs[1,0].set_title('Plot Q-Q')
axs[1,0].annotate("SW p-val: {:.4f}".format(sw_result[1]), xy=(0.05,0.9), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("AD p-val: {:.4f}".format(ad_result[1]), xy=(0.05,0.8), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))

axs[1,0].annotate("DAG p-val: {:.4f}".format(dag_result[1]), xy=(0.05,0.7), xycoords='axes fraction', fontsize=15,
            bbox=dict(boxstyle="round", fc="none", ec="gray", pad=0.6))
# plot
plot_acf(result["CatBoostRegressor"],  lags=35, ax=axs[1,1],color="fuchsia")
axs[1,1].set_title("Autocorrelation");

plt.savefig("../tutorials/catboost_regressor_forecasting__plot_residual_model.png")
plt.close();

![](../tutorials/catboost_regressor_forecasting__plot_residual_model.png)

## **Predict method**

* h (int)		Number of periods to predict.

For more information on the parameters: [you can see it here](https://nixtla.github.io/mlforecast/forecast.html#mlforecast.predict)

In [25]:
forecast_df = mlf.predict(h=30) 
forecast_df

,unique_id,ds,CatBoostRegressor
0,desktop,2019-08-28,73651.451750
1,desktop,2019-08-29,53216.044518
2,desktop,2019-08-30,39463.117771
3,desktop,2019-08-31,31361.277935
4,desktop,2019-09-01,65678.436368
...,...,...,...
85,tablet,2019-09-22,7163.216509
86,tablet,2019-09-23,24157.655639
87,tablet,2019-09-24,14234.434780
88,tablet,2019-09-25,11684.384257


## **Plot prediction**

In [26]:
fig=plot_series(data, forecast_df, max_insample_length=200,engine="matplotlib")
for ax in fig.get_axes():
   ax.set_title("Prediction")
fig.savefig('../tutorials/catboost_regressor_forecasting__plot_forecasting_intervals.png')

![](../figs/catboost_regressor_forecasting__plot_forecasting_intervals.png)

# **Feature importances** <a class="anchor" id="6"></a>

[Table of Contents](#0.1)

Take a look at the feature importances too.

This model is heavily dependent on the lag 1 feature, as you can see in the feature importance plot for `CatBoostRegressor`.

In [27]:
fig=pd.Series(mlf.models_['CatBoostRegressor'].feature_importances_, 
              index=mlf.ts.features_order_).sort_values(ascending=False).plot.bar(title='Feature Importance CatBoostRegressor')
plt.savefig('../tutorials/catboost_regressor_forecasting__plot_feature_importance.png',dpi=300)
plt.close()

![](../tutorials/catboost_regressor_forecasting__plot_feature_importance.png)

# **Evaluate the model’s performance** <a class="anchor" id="7"></a>

[Table of Contents](#0.1)

In previous steps, we’ve taken our historical data to predict the future. However, to asses its accuracy we would also like to know how the model would have performed in the past. To assess the accuracy and robustness of your models on your data perform `Cross-Validation`.

With time series data, `Cross Validation` is done by defining a sliding window across the historical data and predicting the period following it. This form of `cross-validation` allows us to arrive at a better estimation of our model’s predictive abilities across a wider range of temporal instances while also keeping the data in the training set contiguous as is required by our models.

The following graph depicts such a `Cross Validation` Strategy:

![](https://raw.githubusercontent.com/Nixtla/statsforecast/main/nbs/imgs/ChainedWindows.gif)

## **Perform time series cross-validation**

In order to get an estimate of how well our model will be when predicting future data we can perform cross validation, which consist on training a few models independently on different subsets of the data, using them to predict a validation set and measuring their performance.

Since our data depends on time, we make our splits by removing the last portions of the series and using them as validation sets. This process is implemented in `MLForecast.cross_validation`.

`Cross-validation` of time series models is considered a best practice but most implementations are very slow. The `MlForecast` library implements cross-validation as a distributed operation, making the process less time-consuming to perform. If you have big datasets you can also perform `Cross Validation` in a distributed cluster using `Ray, Dask or Spark`.

Depending on your computer, this step should take around 1 min.

The `cross_validation` method from the StatsForecast class takes the following arguments.

* `df:` training data frame

* `h (int):` represents h steps into the future that are being forecasted. In this case, 12 months ahead.

* `step_size (int):` step size between each window. In other words: how often do you want to run the forecasting processes.

* `n_windows(int):` number of windows used for `cross validation`. In other words: what number of forecasting processes in the past do you want to evaluate.

In [28]:
cv_result = mlf.cross_validation(
    data,
    n_windows=5,  # number of models to train/splits to perform
    window_size=30,  # length of the validation set in each window
)

The crossvaldation_df object is a new data frame that includes the following columns:

* `unique_id:` index. If you dont like working with index just run `crossvalidation_df.resetindex()`.
* `ds:` datestamp or temporal index
* `cutoff:` the last datestamp or temporal index for the `n_windows`.
* `y:` true value
* `model:` columns with the model’s name and fitted value.

In [29]:
cv_result

,unique_id,ds,cutoff,y,CatBoostRegressor
0,desktop,2019-03-31,2019-03-30,134535.0,243601.876090
1,mobile,2019-03-31,2019-03-30,181469.0,260446.684174
2,tablet,2019-03-31,2019-03-30,1848.0,12758.507863
3,desktop,2019-04-01,2019-03-30,581078.0,606587.475291
4,mobile,2019-04-01,2019-03-30,432169.0,421531.118840
...,...,...,...,...,...
85,mobile,2019-08-26,2019-07-28,4853.0,181913.372297
86,tablet,2019-08-26,2019-07-28,199.0,25177.085757
87,desktop,2019-08-27,2019-07-28,438.0,204356.732119
88,mobile,2019-08-27,2019-07-28,557.0,192469.588595


In [30]:
def plot_cv(df, df_cv, uid, fname, last_n=24 * 14):
    cutoffs = df_cv.query('unique_id == @uid')['cutoff'].unique()
    fig, ax = plt.subplots(nrows=len(cutoffs), ncols=1, figsize=(14, 6), gridspec_kw=dict(hspace=0.8))
    for cutoff, axi in zip(cutoffs, ax.flat):
        df.query('unique_id == @uid').tail(last_n).set_index('ds').plot(ax=axi, title=uid, y='y')
        df_cv.query('unique_id == @uid & cutoff == @cutoff').set_index('ds').plot(ax=axi, title=uid, y='CatBoostRegressor')
    fig.savefig(fname, bbox_inches='tight')
    plt.close()

In [31]:
plot_cv(data, cv_result, 'desktop', '../tutorials/catboost_regressor_forecasting__cross_validation_predictions.png')

![](../tutorials/catboost_regressor_forecasting__cross_validation_predictions.png)

# **Evaluate the model** <a class="anchor" id="8"></a>

[Table of Contents](#0.1)

We can now compute the accuracy of the forecast using an appropiate accuracy metric. Here we’ll use the Root Mean Squared Error (RMSE). To do this, we first need to `install datasetsforecast`, a Python library developed **by Nixtla** that includes a function to compute the RMSE.

`pip install datasetsforecast`

In [32]:
from datasetsforecast.losses import rmse

The function to compute the RMSE takes two arguments:

1. The actual values.
2. The forecasts, in this case `CatBoostRegressor Model`.

In [33]:
from datasetsforecast.losses import mse, mae, rmse

def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [34]:
evaluation_df = evaluate_cross_validation(cv_result.set_index("unique_id"), rmse)

evaluation_df

,CatBoostRegressor,best_model
unique_id,,
desktop,97095.496667,CatBoostRegressor
mobile,107668.162071,CatBoostRegressor
tablet,13282.232375,CatBoostRegressor


The result obtained in this case is given by each type of `unique_ide` identifying the model created with the `RMSE` metric, in which both results are compared.

In [35]:
cv_rmse = cv_result.groupby(['unique_id', 'cutoff']).apply(lambda df: rmse(df['y'], df['CatBoostRegressor'])).mean()
print("RMSE using cross-validation: ", cv_rmse)

RMSE using cross-validation:  72681.96370426571


# **References** <a class="anchor" id="9"></a>

[Table of Contents](#0)

1. Changquan Huang • Alla Petukhina. Springer series (2022). Applied Time Series Analysis and Forecasting with Python. 
2. Ivan Svetunkov. [Forecasting and Analytics with the Augmented Dynamic Adaptive Model (ADAM)](https://openforecast.org/adam/)
3. [James D. Hamilton. Time Series Analysis Princeton University Press, Princeton, New Jersey, 1st Edition, 1994.](https://press.princeton.edu/books/hardcover/9780691042893/time-series-analysis)
4. [Nixtla Parameters for Mlforecast](https://nixtla.github.io/mlforecast/forecast.html).
5. [Pandas available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).
6. [Rob J. Hyndman and George Athanasopoulos (2018). “Forecasting principles and practice, Time series cross-validation”.](https://otexts.com/fpp3/tscv.html).
7. [Seasonal periods- Rob J Hyndman](https://robjhyndman.com/hyndsight/seasonal-periods/).
8. [Catboost](https://catboost.ai)